In [ ]:
#Removes indeces

import h5py
import numpy as np

coch_file = "/home/ribeirop/OMFOLDER/audiosetDL/unbalanced_stripped/unbalanced_train_segments_Coch.hdf5"
output = "/home/ribeirop/OMFOLDER/audiosetDL/MODIFIED_LABELS/unbalanced_train_segments_nomusic_labels_only.hdf5"

#coch_file = "/home/ribeirop/OMFOLDER/audiosetDL/eval_stripped/eval_segments_Coch.hdf5"
#output = "/home/ribeirop/OMFOLDER/audiosetDL/MODIFIED_LABELS/eval_segments_nomusic_labels_only.hdf5"

coch_data = h5py.File(coch_file)
size = coch_data.attrs.get("size")

newlabelset = h5py.File(output)
labeldata = newlabelset.create_dataset("labels",(size,526),dtype=bool)

print(np.append(coch_data["/labels"][:,0:136],coch_data["/labels"][:,137:527],axis=1))

labeldata[:] = np.append(coch_data["/labels"][:,0:136],coch_data["/labels"][:,137:527],axis=1)

newlabelset.close()
coch_data.close()

print("done")

In [ ]:
#Gets the indeces and saves those as ints
import h5py
import numpy as np
import time

data_file = "/home/ribeirop/OMFOLDER/audiosetDL/balanced_stripped/balanced_train_segments_Coch.hdf5"
output = "/home/ribeirop/OMFOLDER/audiosetDL/MODIFIED_LABELS/balanced_train_segments_label_indeces.hdf5"

data = h5py.File(data_file,"r")

starttime = time.time()
total_count = 0
indeces_list = []
count = 0 
for row in data["/labels"]:
    if(count >= 100):
        print("hey")
        count = 0
        indeces_list = []
    count = count + 1
    indeces= np.where(row==1)[0]
    total_count = total_count + len(indeces)
    indeces_list.append(np.pad(indeces,[0,15-len(indeces)],mode = 'constant',constant_values=-1))
endtime = time.time()

print(endtime-starttime)
    
a = indeces_list[0:10]
print(a)

In [ ]:
#Gets the indeces and saves total count
import h5py
import numpy as np
import time

coch_file = "/home/ribeirop/OMFOLDER/audiosetDL/balanced_stripped/balanced_train_segments_Coch.hdf5"
#output = "/home/ribeirop/OMFOLDER/audiosetDL/MODIFIED_LABELS/balanced_train_segments_label_counts.hdf5"

#coch_file = "/home/ribeirop/OMFOLDER/audiosetDL/unbalanced_stripped/unbalanced_train_segments_Coch.hdf5"
#output = "/home/ribeirop/OMFOLDER/audiosetDL/MODIFIED_LABELS/unbalanced_train_segments_label_counts.hdf5"

#coch_file = "/home/ribeirop/OMFOLDER/audiosetDL/eval_stripped/eval_segments_Coch.hdf5"

coch_data = h5py.File(coch_file, "r")
size = coch_data.attrs.get("size")


starttime = time.time()
total_count = 0
count_list = [None] * size
count = 0
for row in coch_data["/labels"]:
    if(count%1000==0):
        print(count)
    indeces= np.where(row==1)[0]
    
    one_hot = [0] * 15
    one_hot[len(indeces)-1] = 1
    
    count_list[count] = one_hot
    count = count + 1
    
endtime = time.time()

print(endtime-starttime)
#print(count_list)

np.save("/home/ribeirop/OMFOLDER/audiosetDL/MODIFIED_LABELS/balanced_label_counts.npy",np.array(count_list))

coch_data.close()

print("done")

In [ ]:
import h5py
import numpy as np

bl = h5py.File("/home/ribeirop/OMFOLDER/audiosetDL/unbalanced_stripped/unbalanced_train_segments_Coch.hdf5","r")
bc = np.load("/home/ribeirop/OMFOLDER/audiosetDL/MODIFIED_LABELS/unbalanced_label_counts.npy")

print(bc.max())

for index in range(0,10):
    print()
    print(len(np.where(bl["/labels"][index]==True)[0]))
    print(bc[index])


In [ ]:
bc = np.load("/home/ribeirop/OMFOLDER/audiosetDL/MODIFIED_LABELS/balanced_label_counts.npy")
bc[0]

In [3]:
#removes any labels that have a child in the labels
import h5py
import numpy as np
import json
from pprint import pprint
from csv import reader
import csv
def genLabelDictionary(path):
    f = open(path)
    dic = {}
    f.readline()    
    for line in reader(f):
        base = line[1]
        dic[base] = line[0]
        dic[line[0]] = base    
    return dic

def genLabelDictionaryEnglish(path):
    f = open(path)
    dic = {}
    f.readline()    
    for line in reader(f):
        base = line[1]
        dic[base] = line[2]
        dic[line[0]] = line[2]    
    return dic

english_dict = genLabelDictionaryEnglish("/home/ribeirop/mindhiveaudiofolder/class_labels_indices.csv")
indeces_dict = genLabelDictionary("/home/ribeirop/mindhiveaudiofolder/class_labels_indices.csv")
def get_children(idcode,Ontology):
    children = Ontology[idcode]['child_ids']
    #print(children)
    
    if len(children) == 0:
        return children
    else:
        mychildren = children.copy()
        for n in mychildren:
            children = children + get_children(n,Ontology)
    
    return children

def Only_Leaves(data,Ontology,dic):
    newdata = data.copy()
    for i in range(len(data)):
        if data[i] == True:
            children = get_children(dic[str(i)],Ontology)
            for c in children:
                if c in dic:
                    if data[int(dic[c])] == 1:
                        newdata[i] = 0
                        break
    
    return newdata

def get_label_list(dic, l):
    string = ""
    for i in l:
        if str(i) in dic:
            string = string + "[{0}],".format(dic[str(i)])
        else:
            return string
    return string

with open('ontology.json') as data_file:
    data = json.load(data_file)
    Ontology = {}
    for category in data:
        Ontology[category['id']] = category
    english_dict = genLabelDictionaryEnglish("/home/ribeirop/mindhiveaudiofolder/class_labels_indices.csv")
    indeces_dict = genLabelDictionary("/home/ribeirop/mindhiveaudiofolder/class_labels_indices.csv")

    
    #print(type(data))
    #coch_file = "/home/ribeirop/OMFOLDER/audiosetDL/unbalanced_stripped/unbalanced_train_segments_Coch.hdf5"
    #output = "/home/ribeirop/OMFOLDER/audiosetDL/MODIFIED_LABELS/unbalanced_train_segments_labels_leavesonly.hdf5"

    coch_file = "/home/ribeirop/OMFOLDER/audiosetDL/eval_stripped/eval_segments_Coch.hdf5"
    
    coch_data = h5py.File(coch_file,"r")
    size = coch_data.attrs.get("size")
    
    count_list = [None] * size
    count = 0
    maxcount = 0
    for labelrow in coch_data["/labels"]:
        #print("###")
        newrow = Only_Leaves(labelrow,Ontology,indeces_dict)
        #print("original: ",get_label_list(english_dict,np.where(labelrow == True)[0]))
        #print("new: ",get_label_list(english_dict,np.where(newrow == 1)[0]))
       
        if(count%1000==0):
            print(count)
            
        indeces= np.where(newrow==1)[0]
        one_hot = [0] * 11
        one_hot[len(indeces)-1] = 1
        count_list[count] = one_hot
        count = count + 1
        
        if len(indeces) > maxcount:
            maxcount = len(indeces)
            #print(labelrow)
            print("foo", count)

        
    print(maxcount)
    #print(count_list)

    np.save("/home/ribeirop/OMFOLDER/audiosetDL/MODIFIED_LABELS/eval_leaf_label_counts.npy",np.array(count_list))
    
    coch_data.close()

print("done")

0
foo 1
foo 12
foo 31
foo 135
1000
2000
foo 2429
3000
4000
5000
6000
7000
8000
9000
10000
foo 10795
11000
12000
13000
14000
15000
16000
foo 16563
17000
18000
19000
20000
10
done


In [ ]:
english_dict = genLabelDictionaryEnglish("/home/ribeirop/mindhiveaudiofolder/class_labels_indices.csv")
english_dict['180']

In [ ]:
import json
from pprint import pprint
import numpy as np

from csv import reader
import csv
def genLabelDictionary(path):
    f = open(path)
    dic = {}
    f.readline()    
    for line in reader(f):
        base = line[1]
        dic[base] = line[0]
        dic[line[0]] = base
        
        
        print("***")
        print(base)
        print(line[0])
    
    return dic

def genLabelDictionaryEnglish(path):
    f = open(path)
    dic = {}
    f.readline()    
    for line in reader(f):
        base = line[1]
        dic[base] = line[2]
        dic[line[0]] = line[2]
        
        
        print("***")
        print(base)
        print(line[0])
    
    return dic
english_dict = genLabelDictionaryEnglish("/home/ribeirop/mindhiveaudiofolder/class_labels_indices.csv")
indeces_dict = genLabelDictionary("/home/ribeirop/mindhiveaudiofolder/class_labels_indices.csv")

def get_children(idcode,Ontology):
    children = Ontology[idcode]['child_ids']
    #print(children)
    
    if len(children) == 0:
        return children
    else:
        mychildren = children.copy()
        for n in mychildren:
            children = children + get_children(n,Ontology)
    
    return children

def Only_Leaves(data,Ontology,dic):
    newdata = data.copy()
    for i in range(len(data)):
        if data[i] == True:
            children = get_children(dic[str(i)],Ontology)
            for c in children:
                if c in dic:
                    if data[int(dic[c])] == 1:
                        newdata[i] = 0
                        break
    
    return newdata

with open('ontology.json') as data_file:
    jason = json.load(data_file)
    
    Ontology = {}
    for category in jason:
        Ontology[category['id']] = category
    
    #print(type(data))
    print(data[5]['name'])
    
    print(indeces_dict['137'])
    
    allchilds = get_children(indeces_dict['137'],Ontology)
    for c in allchilds:
        if c in english_dict:
            print(english_dict[c])
            print(indeces_dict[c])
            
    a = np.zeros(527)
    a[137] = 1
    a[171] = 1
    a[174] = 1
    
    print(np.where(a==1))
    print(np.where(Only_Leaves(a,Ontology,indeces_dict)))

In [ ]:
from csv import reader
import csv
def genLabelDictionary(path):
    f = open(path)
    dic = {}
    f.readline()    
    for line in reader(f):
        base = line[1]
        dic[base] = line[0]
        dic[line[0]] = base
        
        
        print("***")
        print(base)
        print(line[0])
    
    return dic

def genLabelDictionaryEnglish(path):
    f = open(path)
    dic = {}
    f.readline()    
    for line in reader(f):
        base = line[1]
        dic[base] = line[2]
        dic[line[0]] = line[2]
        
        
        print("***")
        print(base)
        print(line[0])
    
    return dic

english_dict = genLabelDictionaryEnglish("/home/ribeirop/mindhiveaudiofolder/class_labels_indices.csv")
indeces_dict = genLabelDictionary("/home/ribeirop/mindhiveaudiofolder/class_labels_indices.csv")

In [ ]:
indeces_dict['/m/01h8n0']